In [1]:
from collections import Counter, OrderedDict
import os
import shutil
from urllib.parse import urlparse, urljoin
import ftplib
import pandas as pd
import pybel
import json
import sqlalchemy

from tqdm import tqdm_notebook

In [2]:
BASE_URL = 'ftp://ftp.ncbi.nlm.nih.gov/pubchem/'
COMPOUND_FOLDER = 'Compound/CURRENT-Full/SDF' 

In [3]:
parsed_url = urlparse(BASE_URL)

## Get list of .sdf.gz files to download

In [4]:
try:
    with ftplib.FTP() as ftp:
        print(ftp.connect(parsed_url.netloc))
        print(ftp.login())
        print(ftp.cwd(os.path.join(parsed_url.path, COMPOUND_FOLDER)))
        dir_list = ftp.nlst()
        dir_list_sizes = list(ftp.mlsd(facts = ["type", "size"]))
except Exception as e:
    print(e)

220-
 This warning banner provides privacy and security notices consistent with 
 applicable federal laws, directives, and other federal guidance for accessing 
 this Government system, which includes all devices/storage media attached to 
 this system. This system is provided for Government-authorized use only. 
 Unauthorized or improper use of this system is prohibited and may result in 
 disciplinary action and/or civil and criminal penalties. At any time, and for 
 any lawful Government purpose, the government may monitor, record, and audit 
 your system usage and/or intercept, search and seize any communication or data 
 transiting or stored on this system. Therefore, you have no reasonable 
 expectation of privacy. Any communication or data transiting or stored on this 
 system may be disclosed or used for any lawful Government purpose.
220 FTP Server ready.
230 Anonymous access granted, restrictions apply
250 CWD command successful


In [5]:
sorted(dir_list)[:10]

['Compound_000000001_000025000.sdf.gz',
 'Compound_000025001_000050000.sdf.gz',
 'Compound_000050001_000075000.sdf.gz',
 'Compound_000075001_000100000.sdf.gz',
 'Compound_000100001_000125000.sdf.gz',
 'Compound_000125001_000150000.sdf.gz',
 'Compound_000150001_000175000.sdf.gz',
 'Compound_000175001_000200000.sdf.gz',
 'Compound_000200001_000225000.sdf.gz',
 'Compound_000225001_000250000.sdf.gz']

In [6]:
dir_list_sizes[:10]

[('.', {'size': '430080', 'type': 'cdir'}),
 ('..', {'size': '4096', 'type': 'pdir'}),
 ('Compound_000025001_000050000.sdf.gz', {'size': '14923416', 'type': 'file'}),
 ('Compound_000000001_000025000.sdf.gz', {'size': '15603741', 'type': 'file'}),
 ('Compound_000050001_000075000.sdf.gz', {'size': '16000340', 'type': 'file'}),
 ('Compound_000075001_000100000.sdf.gz', {'size': '15272699', 'type': 'file'}),
 ('Compound_000100001_000125000.sdf.gz', {'size': '18039313', 'type': 'file'}),
 ('Compound_000125001_000150000.sdf.gz', {'size': '16504873', 'type': 'file'}),
 ('Compound_000150001_000175000.sdf.gz', {'size': '19788479', 'type': 'file'}),
 ('Compound_000200001_000225000.sdf.gz', {'size': '15571582', 'type': 'file'})]

## Download and process batch

In [7]:
def get_batch(batch_filename):

    # We copy the folder structure of the server
    os.makedirs(COMPOUND_FOLDER, exist_ok=True)
    batch_path = os.path.join(COMPOUND_FOLDER, batch_filename)

    # Download the files
    if not os.path.exists(batch_path):
        try:
            with ftplib.FTP() as ftp, open(batch_path, 'wb') as f:
                print(ftp.connect(parsed_url.netloc))
                print(ftp.login())
                print(ftp.cwd(parsed_url.path))
                print(ftp.retrbinary('RETR ' + batch_path, f.write))
                print('Downloaded file %s (%d MB)' % (batch_path, os.path.getsize(batch_path)//1024//1024))
        except ftplib.all_errors as e:
            print('There was an error with FTP connection')
            print(e)
            
    return batch_path

In [8]:
# OpenBabel is faster and parses all the compounds whereas rdkit is slower and sometimes returns None when he can't parse the mol

def read_batch(batch_path):
    compounds = []
    progress_bar = tqdm_notebook(total=25000, leave=False)
    for i,mol in enumerate(pybel.readfile('sdf', batch_path)):
        if i and i % 100 == 0:
            progress_bar.update(100)
        props = mol.data
        compound_obj = OrderedDict({'c_id':props['PUBCHEM_COMPOUND_CID']})
        compound_obj.update({'inchi':props['PUBCHEM_IUPAC_INCHI'], 'inchi_key':props['PUBCHEM_IUPAC_INCHIKEY']})
        compound_obj.update({'can_smiles':props['PUBCHEM_OPENEYE_CAN_SMILES']})
        if 'PUBCHEM_IUPAC_NAME' in props:
            compound_obj.update({'iupac_name' : props['PUBCHEM_IUPAC_NAME']})
        compounds.append(compound_obj)
    
    progress_bar.update(progress_bar.total)
    progress_bar.close()    
    return compounds

## Run

In [9]:
# Only work with a subset of compounds, in production we will work with all of them
batch_subset = sorted(dir_list)[:1]

In [10]:
all_compounds = []
for batch_filename in tqdm_notebook(batch_subset):
    batch_path = get_batch(batch_filename)
    compounds = read_batch(batch_path)
    all_compounds.extend(compounds)

In [11]:
df = pd.DataFrame(all_compounds)
df.set_index('c_id', inplace=True)
df.head(10)

,inchi,inchi_key,can_smiles,iupac_name
c_id,,,,
1,InChI=1S/C9H17NO4/c1-7(11)14-8(5-9(12)13)6-10(...,RDHQFKQIGNGIED-UHFFFAOYSA-N,CC(=O)OC(CC(=O)[O-])C[N+](C)(C)C,3-acetyloxy-4-(trimethylazaniumyl)butanoate
2,InChI=1S/C9H17NO4/c1-7(11)14-8(5-9(12)13)6-10(...,RDHQFKQIGNGIED-UHFFFAOYSA-O,CC(=O)OC(CC(=O)O)C[N+](C)(C)C,(2-acetyloxy-3-carboxypropyl)-trimethylazanium
3,InChI=1S/C7H8O4/c8-5-3-1-2-4(6(5)9)7(10)11/h1-...,INCSWYKICIYAHB-UHFFFAOYSA-N,C1=CC(C(C(=C1)C(=O)O)O)O,"5,6-dihydroxycyclohexa-1,3-diene-1-carboxylic ..."
4,"InChI=1S/C3H9NO/c1-3(5)2-4/h3,5H,2,4H2,1H3",HXKKHQJGJAFBHI-UHFFFAOYSA-N,CC(CN)O,1-aminopropan-2-ol
5,"InChI=1S/C3H8NO5P/c4-1-3(5)2-9-10(6,7)8/h1-2,4...",HIQNVODXENYOFK-UHFFFAOYSA-N,C(C(=O)COP(=O)(O)O)N,(3-amino-2-oxopropyl) dihydrogen phosphate
6,InChI=1S/C6H3ClN2O4/c7-5-2-1-4(8(10)11)3-6(5)9...,VYZAHLCBVHPDDF-UHFFFAOYSA-N,C1=CC(=C(C=C1[N+](=O)[O-])[N+](=O)[O-])Cl,"1-chloro-2,4-dinitrobenzene"
7,InChI=1S/C7H9N5/c1-2-12-4-11-5-6(8)9-3-10-7(5)...,MUIPLRMGAXZWSQ-UHFFFAOYSA-N,CCN1C=NC2=C1N=CN=C2N,9-ethylpurin-6-amine
8,"InChI=1S/C6H12O4/c1-3-6(2,10)4(7)5(8)9/h4,7,10...",PDGXJDXVGMHUIR-UHFFFAOYSA-N,CCC(C)(C(C(=O)O)O)O,"2,3-dihydroxy-3-methylpentanoic acid"
9,InChI=1S/C6H13O9P/c7-1-2(8)4(10)6(5(11)3(1)9)1...,INAPMGSXUVUWAF-UHFFFAOYSA-N,C1(C(C(C(C(C1O)O)OP(=O)(O)O)O)O)O,"(2,3,4,5,6-pentahydroxycyclohexyl) dihydrogen ..."


## Persist to DB

In [12]:
with open('sql_config.json') as f:
    sql_config = json.load(f)
sql_connection_url = 'postgresql://{user}:{password}@{host}/{dbname}'.format(**sql_config)

In [13]:
engine = sqlalchemy.create_engine(sql_connection_url)

In [14]:
engine.execute('CREATE SCHEMA IF NOT EXISTS %s;' % sql_config['schema'])

In [15]:
# Let pandas and sqlalchemy do the create table and insert queries
df.to_sql(name='compounds', con=engine, schema=sql_config['schema'], if_exists='replace')

In [16]:
engine.execute('ALTER TABLE %s.compounds ADD PRIMARY KEY (c_id)' % sql_config['schema'])

In [17]:
# Verify it's actually there
pd.read_sql('SELECT * FROM bioassay.compounds LIMIT 10', engine)

,c_id,inchi,inchi_key,can_smiles,iupac_name
0,1,InChI=1S/C9H17NO4/c1-7(11)14-8(5-9(12)13)6-10(...,RDHQFKQIGNGIED-UHFFFAOYSA-N,CC(=O)OC(CC(=O)[O-])C[N+](C)(C)C,3-acetyloxy-4-(trimethylazaniumyl)butanoate
1,2,InChI=1S/C9H17NO4/c1-7(11)14-8(5-9(12)13)6-10(...,RDHQFKQIGNGIED-UHFFFAOYSA-O,CC(=O)OC(CC(=O)O)C[N+](C)(C)C,(2-acetyloxy-3-carboxypropyl)-trimethylazanium
2,3,InChI=1S/C7H8O4/c8-5-3-1-2-4(6(5)9)7(10)11/h1-...,INCSWYKICIYAHB-UHFFFAOYSA-N,C1=CC(C(C(=C1)C(=O)O)O)O,"5,6-dihydroxycyclohexa-1,3-diene-1-carboxylic ..."
3,4,"InChI=1S/C3H9NO/c1-3(5)2-4/h3,5H,2,4H2,1H3",HXKKHQJGJAFBHI-UHFFFAOYSA-N,CC(CN)O,1-aminopropan-2-ol
4,5,"InChI=1S/C3H8NO5P/c4-1-3(5)2-9-10(6,7)8/h1-2,4...",HIQNVODXENYOFK-UHFFFAOYSA-N,C(C(=O)COP(=O)(O)O)N,(3-amino-2-oxopropyl) dihydrogen phosphate
5,6,InChI=1S/C6H3ClN2O4/c7-5-2-1-4(8(10)11)3-6(5)9...,VYZAHLCBVHPDDF-UHFFFAOYSA-N,C1=CC(=C(C=C1[N+](=O)[O-])[N+](=O)[O-])Cl,"1-chloro-2,4-dinitrobenzene"
6,7,InChI=1S/C7H9N5/c1-2-12-4-11-5-6(8)9-3-10-7(5)...,MUIPLRMGAXZWSQ-UHFFFAOYSA-N,CCN1C=NC2=C1N=CN=C2N,9-ethylpurin-6-amine
7,8,"InChI=1S/C6H12O4/c1-3-6(2,10)4(7)5(8)9/h4,7,10...",PDGXJDXVGMHUIR-UHFFFAOYSA-N,CCC(C)(C(C(=O)O)O)O,"2,3-dihydroxy-3-methylpentanoic acid"
8,9,InChI=1S/C6H13O9P/c7-1-2(8)4(10)6(5(11)3(1)9)1...,INAPMGSXUVUWAF-UHFFFAOYSA-N,C1(C(C(C(C(C1O)O)OP(=O)(O)O)O)O)O,"(2,3,4,5,6-pentahydroxycyclohexyl) dihydrogen ..."
9,11,InChI=1S/C2H4Cl2/c3-1-2-4/h1-2H2,WSLDOOZREJYCGB-UHFFFAOYSA-N,C(CCl)Cl,"1,2-dichloroethane"
